In [45]:
import numpy as np
import pandas as pd
import string
from tabulate import tabulate

import utility as util
import utility.doc

# utility function
newline = '\n'
alphabet = tuple(k for k in string.ascii_lowercase)
print(alphabet[:10])

def df_to_str(df):
    return tabulate(df, headers='keys', showindex=False, tablefmt='grid')

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j')


In [9]:
# creating a dataframe
pd.DataFrame({'key': alphabet[:4], 'val': 4})

,key,val
0,a,4
1,b,4
2,c,4
3,d,4


In [18]:
# Print dataframe as a table
df = pd.DataFrame({'key': alphabet[:4], 'val': 4})
print(tabulate(df, headers='keys', showindex=False, tablefmt='grid'))

+-------+-------+
| key   |   val |
+=======+=======+
| a     |     4 |
+-------+-------+
| b     |     4 |
+-------+-------+
| c     |     4 |
+-------+-------+
| d     |     4 |
+-------+-------+


In [46]:
# DataFrame basics
df = pd.DataFrame({'key': alphabet[:4], 'val': 4})
util.doc.results(
    "data frame", df_to_str(df),
    "'key' column", df['key'],
    "'val' column", df['val'],
    "columns", df.columns,
)

data frame
+-------+-------+
| key   |   val |
+=======+=======+
| a     |     4 |
+-------+-------+
| b     |     4 |
+-------+-------+
| c     |     4 |
+-------+-------+
| d     |     4 |
+-------+-------+

'key' column
0    a
1    b
2    c
3    d
Name: key, dtype: object

'val' column
0    4
1    4
2    4
3    4
Name: val, dtype: int64

columns
Index(['key', 'val'], dtype='object')


In [49]:
df = pd.DataFrame({'key': alphabet[:4], 'val': range(1,5)})
df

,key,val
0,a,1
1,b,2
2,c,3
3,d,4


In [3]:
# Series basics
s = pd.Series({'key': 'a', 'val': 1})
s['key'], s['val'], s.index, s.values

('a', 1, Index(['key', 'val'], dtype='object'), array(['a', 1], dtype=object))

In [30]:
s = pd.Series({'key': 'a', 'val1': 1, 'val2': 2})
s[['val1', 'val2']].values

array([1, 2], dtype=object)

In [22]:
# Select from the data frame entries where 'key' is 'a'.
df = pd.DataFrame({'key': alphabet[:2] + alphabet[:2], 'val': np.arange(1,5),})
df.loc[df['key'] == 'a']

,key,val
0,a,1
2,a,3


In [23]:
# Select from the data frame entries where 'key' is not 'a'.
df = pd.DataFrame({'key': alphabet[:2] + alphabet[:2], 'val': np.arange(1,5),})
df.loc[df['key'] != 'a']

,key,val
1,b,2
3,b,4


In [27]:
# Sort data frame by values in the 'val' column
df = pd.DataFrame({'key': alphabet[:4], 'val': (2,1,4,3),})
df.sort_values('val')

,key,val
1,b,1
0,a,2
3,d,3
2,c,4


In [32]:
df = pd.DataFrame({'key': alphabet[:4], 'val': (2,1,4,3),})
for s in df.iterrows():
    print(s)

(0, key    a
val    2
Name: 0, dtype: object)
(1, key    b
val    1
Name: 1, dtype: object)
(2, key    c
val    4
Name: 2, dtype: object)
(3, key    d
val    3
Name: 3, dtype: object)


In [33]:
df = pd.DataFrame({'key': alphabet[:4], 'val': (2,1,4,3),})
len(df)

4

In [17]:
# If the data frame has only one entry where 'key' is 'a', then select it as a series.
df = pd.DataFrame({'key': alphabet[:4], 'val': np.arange(1,5),})
df.loc[df['key'] == 'a'].iloc[0]

key    a
val    1
Name: 0, dtype: object

In [5]:
# DataFrame basics

# Create a DataFrame using dict of (str: list)
df = pd.DataFrame({'key': alphabet[:4], 'val': np.arange(1,5),})

# each column or row is a pd.Series
print(type(df['val']), type(df.loc[2]))

# index a DataFrame to retrieve a column from it
print(df['val'], newline)

# use DataFrame.loc label based indexing to retrieve a row from it
print(df.loc[2], newline)

# use DataFrame.iloc integer based indexing to retrieve a row from it
print(df.iloc[-1], newline)

# using DataFrame.head() to summarize data frame
df.head()

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
0    1
1    2
2    3
3    4
Name: val, dtype: int64 

key    c
val    3
Name: 2, dtype: object 

key    d
val    4
Name: 3, dtype: object 



,key,val
0,a,1
1,b,2
2,c,3
3,d,4


In [5]:
# Accessing cells of a data frame by row first, or by column first
df = pd.DataFrame({'key': alphabet[:4], 'val': np.arange(1,5),})
df, df.iloc[1]['val'], df['val'][1]

(  key  val
 0   a    1
 1   b    2
 2   c    3
 3   d    4,
 2,
 2)

Operations on data frames

In [6]:
def get_mock():
    return pd.DataFrame({
            'key': alphabet[:3],
            'val1': np.arange(1,4), 'val2': np.arange(1,4),
            'val3': np.arange(1,4), 'val4': np.arange(1,4)})
get_mock()

,key,val1,val2,val3,val4
0,a,1,1,1,1
1,b,2,2,2,2
2,c,3,3,3,3


In [7]:
df = get_mock()
df['val5'] = 1
df

,key,val1,val2,val3,val4,val5
0,a,1,1,1,1,1
1,b,2,2,2,2,1
2,c,3,3,3,3,1


In [7]:
# Modify
df = get_mock()
df['key'] = 'prefix/' + df['key']
df['key'] += '/suffix'
df

,key,val1,val2,val3,val4
0,prefix/a/suffix,1,1,1,1
1,prefix/b/suffix,2,2,2,2
2,prefix/c/suffix,3,3,3,3


In [8]:
# Operate on columns
_df = get_mock()
# Add values in selected columns in-place
_df['val1'] += 1
_df['val2'] = 1 # or np.array([1,-1])
_df

,key,val1,val2,val3,val4
0,a,2,1,1,1
1,b,3,1,2,2
2,c,4,1,3,3


In [9]:
# Operate on multiple columns at a time
_df = get_mock()
# Add values in selected columns in-place
_df[['val1', 'val2']] += [1,-1] # or np.array([1,-1])
_df[['val3', 'val4']] = [1,-1] # or np.array([1,-1])
_df

,key,val1,val2,val3,val4
0,a,2,0,1,-1
1,b,3,1,1,-1
2,c,4,2,1,-1


In [10]:
# Operate on multiple columns at a time
_df = get_mock()
# Update values in selected columns in place
_df[['val1', 'val2']] = [[1,-1],[-1,1],[1,-1]] # np.array([[1,-1],[-1,1],[1,-1]])
_df

,key,val1,val2,val3,val4
0,a,1,-1,1,1
1,b,-1,1,2,2
2,c,1,-1,3,3


In [11]:
# Operate on rows
df = get_mock()
# Update values in selected row(s)
df[df['key'] == 'a'] = -1
df.loc[1] = 10
df.loc[2] = pd.Series({'key': 'd', 'val1': 1, 'val2': 1, 'val3': 1, 'val4': 1})
df

,key,val1,val2,val3,val4
0,-1,-1,-1,-1,-1
1,10,10,10,10,10
2,d,1,1,1,1


In [12]:
# Operate on multiple rows at a time
df = get_mock()
# Update values in selected row(s)
df.loc[[1,2]] = np.array([['c',1,1,1,1],['d',-1,-1,-1,-1]])
df

,key,val1,val2,val3,val4
0,a,1,1,1,1
1,c,1,1,1,1
2,d,-1,-1,-1,-1


In [13]:
# Operate on cells
# What is the difference between DataFrames loc(), iloc(), at(), iat()
# https://stackoverflow.com/questions/28757389/pandas-loc-vs-iloc-vs-at-vs-iat
# https://stackoverflow.com/questions/37216485/pandas-at-versus-loc
df = get_mock()
df.at[0, 'val1'] = 100
df.loc[1, 'val2'] = 100
df

,key,val1,val2,val3,val4
0,a,100,1,1,1
1,b,2,100,2,2
2,c,3,3,3,3


In [14]:
# Operate on multiple cells at a time
df = get_mock()
# don't use DataFrame.at(), it's much slower.
df.loc[2:4, ('val3','val4')] = 100
df

,key,val1,val2,val3,val4
0,a,1,1,1,1
1,b,2,2,2,2
2,c,3,3,100,100


More operations

In [2]:
def get_mock():
    return pd.DataFrame({
            'key': alphabet[:2] + alphabet[:2],
            'val1': np.arange(1,5), 'val2': np.arange(1,5),
            'val3': np.arange(1,5), 'val4': np.arange(1,5)})
get_mock()

,key,val1,val2,val3,val4
0,a,1,1,1,1
1,b,2,2,2,2
2,a,3,3,3,3
3,b,4,4,4,4


In [4]:
# modify val1 when where key is 'a'
df = get_mock()
df.loc[df['key'] == 'a','val1'] = 100
df

,key,val1,val2,val3,val4
0,a,100,1,1,1
1,b,2,2,2,2
2,a,100,3,3,3
3,b,4,4,4,4


More operations

In [17]:
# Applying numpy operations on series gives back series
df = pd.DataFrame({'key': alphabet[:6], 'deg': np.linspace(0, 2*np.pi, 6),})
np.sin(df['deg']), np.sin(df['deg'].values)

(0    0.000000e+00
 1    9.510565e-01
 2    5.877853e-01
 3   -5.877853e-01
 4   -9.510565e-01
 5   -2.449294e-16
 Name: deg, dtype: float64,
 array([ 0.00000000e+00,  9.51056516e-01,  5.87785252e-01, -5.87785252e-01,
        -9.51056516e-01, -2.44929360e-16]))

In [18]:
# In-place mutation of columns
df = pd.DataFrame({
    'key': alphabet[:3],
    'val1': np.arange(1,4), 'val2': np.arange(1,4),
    'val3': np.arange(1,4), 'val4': np.arange(1,4)})
df1 = df.copy()
df['val2'] /= 2
df[['val1', 'val3']] *= 2
df['val4'] = np.exp(df['val4'])
df2 = df.copy()
df[['val1', 'val2']] += np.array([1, 2])
df1, df2, df

(  key  val1  val2  val3  val4
 0   a     1     1     1     1
 1   b     2     2     2     2
 2   c     3     3     3     3,
   key  val1  val2  val3       val4
 0   a     2   0.5     2   2.718282
 1   b     4   1.0     4   7.389056
 2   c     6   1.5     6  20.085537,
   key  val1  val2  val3       val4
 0   a     3   2.5     2   2.718282
 1   b     5   3.0     4   7.389056
 2   c     7   3.5     6  20.085537)

In [19]:
# In-place mutation of columns
df = pd.DataFrame({'key': alphabet[:3], 'val1': np.arange(1,4),
        'val2': np.arange(1,4), 'val3': np.arange(2,5), 'val4': np.arange(2,5)})
df[['val1', 'val3']] = 0
df[['val2', 'val4']] = df[['val2', 'val4']].values[0]
df

,key,val1,val2,val3,val4
0,a,0,1,0,2
1,b,0,1,0,2
2,c,0,1,0,2


In [20]:
# Concatenate two data frames.
df1 = pd.DataFrame({'key': alphabet[:4], 'val': np.arange(1,5)})
df2 = pd.DataFrame({'key': alphabet[4:8], 'val': np.arange(5,9)})
pd.concat((df1, df2), ignore_index=True)

,key,val
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6
6,g,7
7,h,8


In [21]:
# Iterate the rows of a data frame.
df = pd.DataFrame({'key': alphabet[:3], 'val': np.arange(1,4)})
for i, s in df.iterrows():
    print(i, *[ f"  {k}: {s[k]}  " for k in s.index ])

0   key: a     val: 1  
1   key: b     val: 2  
2   key: c     val: 3  


In [22]:
# Create an empty DataFrame
columns = ['key', 'val1', 'val2']
df = pd.DataFrame(columns=columns)
print(df, newline)
# append rows to the end of data frame
df.loc[len(df)] = ['a', 1, 1]
df.loc[len(df)] = ['b', 2, 2]
print(df, newline)
# add a new column to DataFrame and populate it
df['str'] = ['abba', 'zetta']
print(df)

Empty DataFrame
Columns: [key, val1, val2]
Index: [] 

  key val1 val2
0   a    1    1
1   b    2    2 

  key val1 val2    str
0   a    1    1   abba
1   b    2    2  zetta


In [23]:
# Convert a series into a data frame
s = pd.Series({'key': 'a', 'val': 1})
pd.DataFrame([s])

,key,val
0,a,1


In [24]:
# Finding unique values of a column in a data frame.
# Create a DataFrame using dict of (str: list)
df = pd.DataFrame({'key': alphabet[:8], 'val': np.arange(1,9),})
df['val'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [25]:
# Select rows in data frame by column values.
data = {'key': alphabet[:8],
        'val': [1, 2, 2, 4, 2, 6, 7, 8]}
df = pd.DataFrame(data)
df[df['val'] == 2]

,key,val
1,b,2
2,c,2
4,e,2


In [26]:
# Merge two data frames (similar to SQL JOIN statement)
# In this use case, I'm using the second data frame to filter
# the first data frame by composite key (frame_id, node_id) 

# See:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

data1 = {
    'frame_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'node_id':  [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'x':        [1, 0, 1, 0, 1, 0, 1, 0, 1],
    'y':        [1, 1, 0, 0, 1, 1, 0, 0, 1],
}
df1 = pd.DataFrame(data1)
data2 = {
    'frame_id': [1, 2, 2, 2, 3, 3],
    'node_id':  [1, 1, 2, 3, 2, 3]
}
df2 = pd.DataFrame(data2)
pd.merge(df1, df2, how='inner', on=['frame_id', 'node_id'])

,frame_id,node_id,x,y
0,1,1,1,1
1,2,1,0,0
2,2,2,1,1
3,2,3,0,1
4,3,2,0,0
5,3,3,1,1


In [27]:
# Example of data retrieval with 'composite keys'
data = {
    'frame_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'node_id':  [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'x':        [1, 0, 1, 0, 1, 0, 1, 0, 1],
    'y':        [1, 1, 0, 0, 1, 1, 0, 0, 1],
}
df = pd.DataFrame(data)
s = df[np.logical_and(df['frame_id'] == 1, df['node_id'] == 2)].iloc[0]
s['x']

0

In [28]:
data = {
    'node_id':  [1, 2, 3],
    'x':        [1, 0, 1],
    'y':        [1, 1, 0],
}
df = pd.DataFrame(data)
# Get the values of column x and y as ndarray.
print(df[['x', 'y']].values)
df[['x', 'y']]

[[1 1]
 [0 1]
 [1 0]]


,x,y
0,1,1
1,0,1
2,1,0


In [29]:
# Make a DataFrame from a dict of iter where iterable items are 'variables' with associated dict key as 'key'.
data = {'animal':  set(['dog','cat','fish']),
        'color':   ['red','blue','green','yellow'],
        'nothing': [],
        'vehicle': iter(['submarine', 'racecar', 'helicopter'])}

df = pd.DataFrame.from_dict(data, orient='index')
print(df)
df = df.stack().to_frame().reset_index().drop('level_1', axis=1)
df.columns = ['key', 'variable']
df

                 0        1           2       3
animal         dog      cat        fish    None
color          red     blue       green  yellow
nothing       None     None        None    None
vehicle  submarine  racecar  helicopter    None


,key,variable
0,animal,dog
1,animal,cat
2,animal,fish
3,color,red
4,color,blue
5,color,green
6,color,yellow
7,vehicle,submarine
8,vehicle,racecar
9,vehicle,helicopter


In [30]:
# Create an empty DataFrame with columns, and default int row indices
columns = ['key', 'val1', 'val2']
data = [['a', 1, 1], ['b', 2, 2], ['a', 1, 1]]
df = pd.DataFrame(data, columns=columns)
print(df)
print()

# selecting rows with key column a
print("Selecting rows where key column contains 'a'")
print(df[df['key'] == 'a'])
print("Number of rows:", len(df[df['key'] == 'a']))
print(isinstance(df[df['key'] == 'a'], pd.DataFrame))
print("'a' in df['key']?", 'a' in df['key'].values )
print()

# selecting rows with key column c
print("Selecting rows where key column contains 'c'")
print("Gives empty data frame")
print(df[df['key'] == 'c'])
print("Number of rows:", len(df[df['key'] == 'c']))
print(isinstance(df[df['key'] == 'c'], pd.DataFrame))
print("'c' in df['key']?", 'c' in df['key'].values )
print()

#
print(df[df['key'] == 'a'].iloc[0])

  key  val1  val2
0   a     1     1
1   b     2     2
2   a     1     1

Selecting rows where key column contains 'a'
  key  val1  val2
0   a     1     1
2   a     1     1
Number of rows: 2
True
'a' in df['key']? True

Selecting rows where key column contains 'c'
Gives empty data frame
Empty DataFrame
Columns: [key, val1, val2]
Index: []
Number of rows: 0
True
'c' in df['key']? False

key     a
val1    1
val2    1
Name: 0, dtype: object


In [31]:
# Create an empty DataFrame with columns, and str row indices
columns = ['val1', 'val2']
df = pd.DataFrame(columns=columns)
print(df)
# append a row to the end of DataFrame
df.loc['a'] = [1, 1]
df.loc['b'] = [2, 2]
df['str'] = ['abba', 'zetta']
print(df)
print("check that ")
print("Index 'b' exists?", 'b' in df.index)
print('c' in df.index)
try:
    df['c']
except KeyError as e:
    print('KeyError', e)
print()

print("Locate a row in a data frame")
print(df.loc['a'])
print()

print("Locate a column in a data frame")
print(df['str'])
print(df['str'].at['a'])
print()

print("Locate a single cell in a data frame")
# DataFrame.at() works similarly to DataFrame.loc()
# can only be used to access cells, not entire axes
print(df.at['a', 'val2'])
print(df.loc['a', 'val2'])
print(df.loc['a'].at['val1'])
print(df.loc['a'].loc['val1'])

Empty DataFrame
Columns: [val1, val2]
Index: []
  val1 val2    str
a    1    1   abba
b    2    2  zetta
check that 
Index 'b' exists? True
False
KeyError 'c'

Locate a row in a data frame
val1       1
val2       1
str     abba
Name: a, dtype: object

Locate a column in a data frame
a     abba
b    zetta
Name: str, dtype: object
abba

Locate a single cell in a data frame
1
1
1
1


In [32]:
# Create a total row at the bottom.
columns = ['val1', 'val2']
df = pd.DataFrame(columns=columns)
df.loc['a'] = [1, 1]
df.loc['b'] = [2, 3]
df.loc['c'] = [3, 9]
# DataFrame.sum() as pd.Series
print(df.sum().astype(int))
print()
df.loc["Total"] = df.sum().astype(int)
print(df)

val1     6
val2    13
dtype: int64

      val1 val2
a        1    1
b        2    3
c        3    9
Total    6   13


In [33]:
# Create an empty DataFrame with columns, and str row indices
columns = ['val1', 'val2', 'str']
data = [[1, 3, 'abba'], [2, 4, 'zetta']]
index = ['a', 'b']
df = pd.DataFrame(data, columns=columns, index=index)
print(df)
df.at['c', 'val1'] = 6
df.at['c', 'val2'] = 7
df.at['c', 'str'] = 'qwerty'
print(df)

   val1  val2    str
a     1     3   abba
b     2     4  zetta
   val1  val2     str
a   1.0   3.0    abba
b   2.0   4.0   zetta
c   6.0   7.0  qwerty


In [34]:
# Select part of the dataframe that contains consecutively increasing frame_id
def longest_sequence_using_split(l, split):
    n = len(l)
    longest_size  = 0
    longest_begin = None
    longest_end   = None
    curr_begin = 0
    curr_end   = 0
    while curr_end < n:
        if split(l, curr_end) or curr_end == n - 1:
            if (curr_end - curr_begin + 1)  > longest_size:
                longest_size  = curr_end - curr_begin + 1
                longest_begin = curr_begin
                longest_end   = curr_end
            curr_begin = curr_end + 1
            curr_end   = curr_end + 1
        else:
            curr_end  += 1
    if longest_size == 0:
        return None, longest_size
    else:
        return slice(longest_begin, longest_end+1), longest_size

def longest_consecutive_increasing_subsequence(l):
    def split(l, i):
        try:
            return l[i + 1] != l[i] + 1
        except:
            return False
    return longest_sequence_using_split(l, split)

data = {
    'frame_id': [1, 2, 3, 4, 5, 6, 8, 9, 10],
    'x':        [1, 0, 1, 0, 1, 0, 1, 0, 1],
    'y':        [1, 1, 0, 0, 1, 1, 0, 0, 1],
    'dx':       [1, 1, 1, 0, 0, 0, 1, 1, 1],
    'dy':       [1, 1, 1, 1, 0, 0, 0, 0, 1],}
df = pd.DataFrame(data)
s, sz = longest_consecutive_increasing_subsequence(df['frame_id'])
print(sz)
df[s]

6


,frame_id,x,y,dx,dy
0,1,1,1,1,1
1,2,0,1,1,1
2,3,1,0,1,1
3,4,0,0,0,1
4,5,1,1,0,0
5,6,0,1,0,0


In [35]:
data1 = {
    'frame_id': [1, 2, 3, 4, 5, 6, 8, 9, 10],
    'x':        [1, 0, 1, 0, 1, 0, 1, 0, 1],
    'y':        [1, 1, 0, 0, 1, 1, 0, 0, 1],
    'dx':       [1, 1, 1, 0, 0, 0, 1, 1, 1],
    'dy':       [1, 1, 1, 1, 0, 0, 0, 0, 1],}
data2 = {
    'frame_id': [0, 1, 2, 5, 6, 7, 8, 9, 10],
    'x':        [1, 0, 1, 0, 1, 0, 1, 0, 1],
    'y':        [1, 1, 0, 0, 1, 1, 0, 0, 1],
    'dx':       [1, 1, 1, 0, 0, 0, 1, 1, 1],
    'dy':       [1, 1, 1, 1, 0, 0, 0, 0, 1],}
df = pd.DataFrame(data2)
print(type(df['frame_id'].values))
s, sz = util.longest_consecutive_increasing_subsequence(df['frame_id'].values)
print(sz)
df = df[s]
# df['frame_id'] = np.arange(sz)
df

<class 'numpy.ndarray'>
6


,frame_id,x,y,dx,dy
3,5,0,0,0,1
4,6,1,1,0,0
5,7,0,1,0,0
6,8,1,0,1,0
7,9,0,0,1,0
8,10,1,1,1,1


In [36]:
# DataFrames sometimes keep views of array data instead of creating a copy of the data
# so modifying the data frame may modify the original reference to the array that was used to create the dataframe.
a  = np.arange(1,4)
b  = a.copy()

example = True
if example:
    df = pd.DataFrame(a, columns=['val'])
else:
    df = pd.DataFrame({
            'key': alphabet[:3],
            'val': a})

df.at[0, 'val'] = 1e12
print(a, b)
df

[1000000000000             2             3] [1 2 3]


,val
0,1000000000000
1,2
2,3


In [6]:
# get mean and deviation from values in data frame corresponding to specific keys
d = {'key': tuple('ababa'), 'val': [-1,0,1,2,100]}
df = pd.DataFrame(d)
float(df[df['key'] == 'a'].mean()), float(np.sqrt( df[df['key'] == 'a'].var() ))

(33.333333333333336, 57.74368652357877)